In [ ]:
#This is for Flowdesk Interview Technical Test 
#3 questions 

1. 
def calculate_pnl(trades):
    """
    trades: list[str], each "stock;price;qty;side;trade_type"
            e.g. "GOOG;2,723.68;42;buy;make"
                 "GOOG;2,792.19;42;sell;take"
    Returns: float rounded to 2 decimals (Net P&L including fees).
    """
    MAKE_FEE = 0.001   # 0.1%
    TAKE_FEE = 0.005   # 0.5%

    # FIFO inventory: list of [buy_price, remaining_qty]
    inventory = []
    realized_pnl = 0.0
    total_fees = 0.0

    for rec in trades:
        # Parse "stock;price;qty;side;trade_type"
        stock, price_str, qty_str, side, trade_type = rec.split(";")

        # Allow comma-separated prices like "2,723.68"
        price = float(price_str.replace(",", ""))
        qty = float(qty_str)

        # Fee for this transaction
        fee_rate = MAKE_FEE if trade_type.lower() == "make" else TAKE_FEE
        total_fees += fee_rate * price * qty

        if side.lower() == "buy":
            # Add to FIFO inventory
            inventory.append([price, qty])
        else:  # sell
            remaining = qty
            while remaining > 0 and inventory:
                buy_price, buy_qty = inventory[0]
                matched = min(remaining, buy_qty)
                realized_pnl += (price - buy_price) * matched

                buy_qty -= matched
                remaining -= matched
                if buy_qty == 0:
                    inventory.pop(0)
                else:
                    inventory[0][1] = buy_qty

            # If remaining > 0 here, it would be a short sale with no inventory.
            # The problem examples always balance buys and sells, so we ignore that case.

    net_pnl = realized_pnl - total_fees
    return round(net_pnl, 2)






In [ ]:
#2. Calculate
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

INPUT_CSV = r"C:\Users\Viriya Gunawan Lim\Downloads\73774c34-270f-4c90-aaca-9b268d5330e2.csv"
OUT_DIR = Path("./canditech_out"); OUT_DIR.mkdir(exist_ok=True, parents=True)
DATE_START = pd.Timestamp("2020-06-01"); DATE_END = pd.Timestamp("2021-05-31")

def round_half_away_from_zero(x):
    s = np.sign(x); return (np.floor(np.abs(x)+0.5)*s).astype(int)

df0 = pd.read_csv(INPUT_CSV); df0.columns=[c.strip().lower() for c in df0.columns]
date_col="date" if "date" in df0.columns else [c for c in df0.columns if "date" in c][0]
close_col="close" if "close" in df0.columns else [c for c in df0.columns if "close" in c][0]

df0[date_col]=pd.to_datetime(df0[date_col], dayfirst=True, errors="coerce")
df0[close_col]=(df0[close_col].astype(str).str.replace(",","",regex=False).str.replace(" ","",regex=False))
df0[close_col]=pd.to_numeric(df0[close_col], errors="coerce")
df0=(df0.dropna(subset=[date_col,close_col]).sort_values(date_col).drop_duplicates(subset=[date_col], keep="last").reset_index(drop=True))
df=df0[(df0[date_col]>=DATE_START)&(df0[date_col]<=DATE_END)].copy().reset_index(drop=True)

df["close"]=df[close_col]
df["close_pct_change"]=df["close"].pct_change().mul(100.0)
if len(df)>0: df.loc[df.index[0], "close_pct_change"]=0.0
df["pct_change_decimal"]=df["close_pct_change"]/100.0
df["abs_integer_change"]=round_half_away_from_zero(df["pct_change_decimal"]*100).abs()
df["is_trade"]=(df["abs_integer_change"]>0)&(df["abs_integer_change"]%2==0)

df["next_close"]=df["close"].shift(-1)
ret=(df["next_close"]-df["close"])/df["close"]
df["profit_percent"]=np.where(df["is_trade"]&ret.notna(), ret*100.0, 0.0)
df["total_profit_percent"]=df["profit_percent"].cumsum()

OUT_CSV=OUT_DIR/"csv final.csv"; df.to_csv(OUT_CSV, index=False)
plt.figure(figsize=(9,4.8)); plt.plot(df[date_col], df["total_profit_percent"]); plt.grid(True); plt.tight_layout()
plt.savefig(OUT_DIR/"total profit graph.png"); plt.close()
print("Done:", OUT_CSV)


In [ ]:
#3. SQL Query

SELECT user_id
FROM abandonment
WHERE YEAR(abandon_date) = 2019
  AND MONTH(abandon_date) = 12;


SELECT first_name, last_name
FROM users
ORDER BY spend DESC
LIMIT 1 OFFSET 1;


SELECT first_name
FROM users
WHERE LENGTH(last_name) = 5
  AND last_name LIKE '%n';


SELECT u.first_name
FROM users u
JOIN abandonment a
  ON u.id = a.user_id
WHERE a.minutes_spent > 70
  AND a.minutes_spent < 80;

#WITHOUT USING TOP or LIMIT METHOD. 
SELECT first_name, last_name, spend
FROM (
    SELECT first_name, last_name, spend,
           RANK() OVER (ORDER BY spend DESC) AS rnk
    FROM users
) ranked
WHERE rnk <= 3;

